In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import required modules

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input,Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Input Data and Data Argumentation 

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                fill_mode='nearest')

valid_gen=ImageDataGenerator(rescale=1./255)

batch_size=32
base_dir="../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"

training_set=train_datagen.flow_from_directory(base_dir+'/train',
                                              target_size=(224,224),
                                              batch_size=batch_size,
                                              class_mode='categorical')

valid_set=valid_gen.flow_from_directory(base_dir+'/valid',
                                       target_size=(224,224),
                                       batch_size=batch_size,
                                       class_mode='categorical')

In [4]:
train_num=training_set.samples
valid_num=valid_set.samples

# Transfer Learning with VGG

Download VGG-16 model without Dense layers

In [5]:
base_model=tf.keras.applications.VGG16(include_top=False,
                                      weights='imagenet',
                                      input_shape=(224,224,3))
base_model.summary()

Freeze the convolutional base created and use its as a feature extractor (by base_model.trainable=False) prevent the weights in a given layer from being updated during training

In [6]:
base_model.trainable=False

In [7]:
base_model.summary()

Build a model by channing the base_model layer and Dense layers

In [8]:
inputs=Input(shape=(224,224,3))
x=base_model(inputs,training=False)
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
outputs=Dense(38,activation='softmax')(x)


vgg_model=Model(inputs,outputs)

vgg_model.summary()

Compile the model before training it. Since there are two classes, use a categorical cross-entropy loss & Adam optimizer

In [9]:

vgg_model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
len(vgg_model.trainable_variables)

In [11]:
vgg_model.evaluate(valid_set)

# Train the model

In [12]:
weightpath='best_weights_9.hdf5'
checkpoint = ModelCheckpoint(weightpath, monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=True, mode='max')

history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,c
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

In [13]:
history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

In [14]:
history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

# Fine Tuning

In the feature extraction experiment, you were only training a few layers on top of an VGG-16 base model. The weights of the pre-trained network were not updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

 The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

In [15]:
base_model.trainable=True #Un-Freezing the base model
vgg_model.summary()

In [16]:
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

fine_tune_history=vgg_model.fit_generator(training_set,
                                        steps_per_epoch=150,
                                        validation_data=valid_set,
                                        epochs=10,
                                        validation_steps=100)

In [1]:
import numpy as np

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input,Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
new_model = tf.keras.models.load_model('../models/')